In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LassoCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc,recall_score,precision_score
from sklearn import metrics

import sys, os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import math
import pickle

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import StratifiedKFold

In [3]:
#https://github.com/dhaitz/mplcyberpunk
import mplcyberpunk

In [4]:
train_time = 1 * 60 * 60
objective = 'lassocvregressor'

In [5]:
sys.path.append(os.path.realpath('..'))

In [6]:
train_df = pd.read_csv(r"..\data\train.csv", index_col=0)
test_df = pd.read_csv(r"..\data\test.csv", index_col=0)
sample_sub = pd.read_csv(r"..\data\sample_submission.csv", index_col=0)

In [7]:
del train_df['Over18']
del train_df['EmployeeCount']
del train_df['StandardHours']

In [8]:
columns_to_vectorize = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']
for vector_target in columns_to_vectorize:
    print(vector_target)
    vectorizer = CountVectorizer()
    vectorizer.fit_transform(train_df[vector_target])
    train_df[f'{vector_target}_v'] = vectorizer.transform(train_df[vector_target]).toarray().argmax(axis=1)[:,None]
    vectorizer.fit_transform(test_df[vector_target])
    test_df[f'{vector_target}_v'] = vectorizer.transform(test_df[vector_target]).toarray().argmax(axis=1)[:,None]

BusinessTravel
Department
EducationField
Gender
JobRole
MaritalStatus
OverTime


In [9]:
features = ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction','HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
        'BusinessTravel_v', 'Department_v', 'EducationField_v','Gender_v', 'JobRole_v', 'MaritalStatus_v', 'OverTime_v']
target = ['Attrition']

In [10]:
scaler = MinMaxScaler().fit(train_df[features])
X = scaler.transform(train_df[features])
X_test = scaler.transform(test_df[features])

scaler = MinMaxScaler().fit(train_df[target])
Y = scaler.transform(train_df[target])

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, stratify=Y, test_size=0.4)

In [12]:
len(X_train)

1006

In [13]:
RANDOM_STATE = 12 
FOLDS = 5
param_grid_history = {}

def render_model(param_grid):
    reg = LassoCV(precompute="auto",
                       fit_intercept=True,
                       max_iter=10000,
                       verbose=False,
                       tol=0.0001,
                       **param_grid,
                       n_jobs=-1)
    return reg
    
def train_model(reg, X_train, y_train, X_valid, y_valid):
    reg.fit(X_train, y_train)

    preds = reg.predict(train_df[features])
    fpr, tpr, _ = roc_curve(train_df[target], preds)
    roc_auc = auc(fpr, tpr)
    return reg, roc_auc

def objective_v2(trial):
    scores = []
    param_grid = {
       'eps':trial.suggest_float('eps', 0.00001, 0.001),
       #'cv':trial.suggest_int('cv', 2, 10),
       'n_alphas':trial.suggest_int('n_alphas', 10, 1000),
    } 
        
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
        print(10*"=", f"Fold={fold+1}", 10*"=")
        X_train, X_valid = train_df.iloc[train_idx][features], train_df.iloc[valid_idx][features]
        y_train , y_valid = train_df[target].iloc[train_idx] , train_df[target].iloc[valid_idx] 
        
        reg = render_model(param_grid)
        reg, roc_auc = train_model(reg, X_train, y_train, X_valid, y_valid)

        scores.append(roc_auc)
    mean_scores = np.mean(scores)
    param_grid_history[mean_scores] = param_grid

    return mean_scores

In [14]:
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name=objective)
study.optimize(objective_v2, timeout=train_time)

[I 2023-01-18 20:05:44,555] A new study created in memory with name: lassocvregressor


========== Fold=1 ==========


D:\source\repos\venv\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1561: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


========== Fold=2 ==========


D:\source\repos\venv\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1561: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

========== Fold=5 ==========


D:\source\repos\venv\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1561: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2023-01-18 20:07:05,789] Trial 56 finished with value: 0.6665003385240352 and parameters: {'eps': 0.0006876766012009476, 'n_alphas': 484}. Best is trial 47 with value: 0.830926201760325.
D:\source\repos\venv\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1561: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


========== Fold=1 ==========


[W 2023-01-18 20:07:06,039] Trial 57 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "D:\source\repos\venv\Python310\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\ryans\AppData\Local\Temp\ipykernel_36992\1584958140.py", line 38, in objective_v2
    reg, roc_auc = train_model(reg, X_train, y_train, X_valid, y_valid)
  File "C:\Users\ryans\AppData\Local\Temp\ipykernel_36992\1584958140.py", line 16, in train_model
    reg.fit(X_train, y_train)
  File "D:\source\repos\venv\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py", line 1661, in fit
    mse_paths = Parallel(
  File "D:\source\repos\venv\Python310\lib\site-packages\joblib\parallel.py", line 1098, in __call__
    self.retrieve()
  File "D:\source\repos\venv\Python310\lib\site-packages\joblib\parallel.py", line 975, in retrieve
    self._output.extend(job.get(timeout=self.timeout))
  Fi

KeyboardInterrupt: 

In [15]:
parm_grid_bytes = pickle.dumps(param_grid_history)
with open(f"rendered_data/{objective}_bytes.hex", "wb") as binary_file:
    binary_file.write(parm_grid_bytes)

In [18]:
def train(param_grid):
    reg = render_model(param_grid)
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(train_df[features], train_df[target])):
        print(10*"=", f"FINAL TRAINING Fold={fold+1}", 10*"=")
        X_train, X_valid = train_df.iloc[train_idx][features], train_df.iloc[valid_idx][features]
        y_train , y_valid = train_df[target].iloc[train_idx] , train_df[target].iloc[valid_idx] 
        reg, roc_auc = train_model(reg, X_train, y_train, X_valid, y_valid)
    return reg

percent = math.ceil(len(param_grid_history.keys()) * .1)
top = sorted(list(param_grid_history.keys()))[-1:]

train_preds = []
test_preds = []
for key in tqdm(top):
    model = train(param_grid_history[key])
    train_preds.append(model.predict(train_df[features]))
    test_preds.append(model.predict(test_df[features]))

train_final_preds = np.stack(train_preds).mean(0)
test_final_preds = np.stack(test_preds).mean(0)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]D:\source\repos\venv\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1561: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


========== FINAL TRAINING Fold=1 ==========
========== FINAL TRAINING Fold=2 ==========
========== FINAL TRAINING Fold=3 ==========


D:\source\repos\venv\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1561: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\source\repos\venv\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1561: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


========== FINAL TRAINING Fold=4 ==========
========== FINAL TRAINING Fold=5 ==========


D:\source\repos\venv\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1561: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\source\repos\venv\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1561: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.02s/it]


In [19]:
train_submission = pd.DataFrame(data={'id': train_df.index, target[0]: train_final_preds})
train_submission.to_csv(fr'rendered_data/{objective}_train_submission.csv', index=False)

test_submission = pd.DataFrame(data={'id': test_df.index, target[0]: test_final_preds})
test_submission.to_csv(fr'rendered_data/{objective}_test_submission.csv', index=False)